In [18]:
import numpy as np
import pandas as pd
import math
from datetime import datetime
from dateutil.relativedelta import relativedelta

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

### Rossmann Store Sales 데이터셋
+ Rossmann: 유럽에 4000개의 체인점과 56,200 명의 종업원을 운영하고 있는 약국 프랜차이즈

https://www.kaggle.com/c/rossmann-store-sales

In [2]:
store = pd.read_csv('./data/store.csv', 
                    dtype={'CompetitionDistance': 'Int32',
                          'CompetitionOpenSinceMonth': 'Int32',
                          'CompetitionOpenSinceYear': 'Int32',
                          'Promo2SinceWeek': 'Int32',
                          'Promo2SinceYear': 'Int32'})


train = pd.read_csv('./data/train.csv')

C:\Dev\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### store.csv 변수
- Store: 스토어 ID
- StoreType: 스토어 유형(a, b, c, d)
- Assortment: 판매 상품 규모에 따른 스토어 유형(a = basic, b = extra, c = extended)
- CompetitionDistance: 주변 경쟁 약국과의 거리
- CompetitionOpenSinceMonth: 주변 경쟁 약국의 개업 월
- CompetitionOpenSinceYear: 주변 경쟁 약국의 개업 년도
- Promo2: 프로모션 행사 참여 여부(0: 미참여, 1: 참여)
- Promo2SinceWeek: 프로모션 참여 시작 주차
- Promo2SinceYear: 프로모션 참여 시작 년도
- PromoInterval: 정기 프로모션 월(예: 2월, 5월, 8월, 11월)

In [3]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270,9,2008,0,<NA>,<NA>,NaN
1,2,a,a,570,11,2007,1,13,2010,"Jan,Apr,Jul,Oct"
2,3,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct"
3,4,c,c,620,9,2009,0,<NA>,<NA>,NaN
4,5,a,a,29910,4,2015,0,<NA>,<NA>,NaN


### train.csv 변수
- Store: 스토어 ID
- DayOfWeek: 요일
- Date: 일자
- Sales: 일별 매출
- Customers: 일별 손님 수
- Open: 해당일자 영업 여부
- Promo: 해당일자 프로모션 여부
- StateHoliday: 공휴일(a: 공휴일, b: 부활절, c: 크리스마스, 0: None)
- SchoolHoliday: 학교 방학일 여부(1: 방학, 0: 그 외)

In [4]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [5]:
"""
    StateHoliday 변수 전처리
    - 원본 값 유형: ['0', 'a', 'b', 'c', 0]
    - 정수 0을 문자 0으로 대체
"""

train.StateHoliday.replace(0, '0', inplace=True)

In [6]:
# 결측치 처리: CompetitionDistance, 처리기법: 중앙값 대체

store['CompetitionDistance'].fillna(store['CompetitionDistance'].median(), inplace=True)
store.fillna(0, inplace=True)

# 변수 데이터 타입 변환: Int32 -> int64 (pandas 처리를 위해)
store = store.astype({'CompetitionDistance' : 'int64',
             'CompetitionOpenSinceMonth' : 'int64',
             'CompetitionOpenSinceYear' : 'int64',
             'Promo2SinceWeek' : 'int64',
             'Promo2SinceYear' : 'int64'}, errors='raise')

store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270,9,2008,0,0,0,0
1,2,a,a,570,11,2007,1,13,2010,"Jan,Apr,Jul,Oct"
2,3,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct"
3,4,c,c,620,9,2009,0,0,0,0
4,5,a,a,29910,4,2015,0,0,0,0


### 학습 데이터 준비
대상 변수: Sales

특징:
+ DayOfWeek, Year, Month, Promo, StateHoliday, SchoolHoliday, StoreType, Assortment, CompetitionDistance, CompetitionOpenElapsedDays, Promo2ElapsedDays, PromoInterval

특징 추출:
+ Year, Month, CompetitionOpenElapsedDays, Promo2ElapsedDays
    + CompetitionOpenElapsedDays: 경쟁 약국 개업일로부터의 경과 시간(일자)
    + Promo2ElapsedDays: 프로모션 시작일로부터의 경과 시간(일자)

인코딩 대상:
+ DayOfWeek, Year, Month, StoreType, Assortment, PromoInterval, StateHoliday
      


In [7]:
# 데이터셋 조인: train + store

train_merged = pd.merge(train, store, left_on='Store', right_on='Store')
train_merged.iloc[0]

Store                                 1
DayOfWeek                             5
Date                         2015-07-31
Sales                              5263
Customers                           555
Open                                  1
Promo                                 1
StateHoliday                          0
SchoolHoliday                         1
StoreType                             c
Assortment                            a
CompetitionDistance                1270
CompetitionOpenSinceMonth             9
CompetitionOpenSinceYear           2008
Promo2                                0
Promo2SinceWeek                       0
Promo2SinceYear                       0
PromoInterval                         0
Name: 0, dtype: object

In [8]:
# Date 로부터 Year, Month 추출
train_merged['Date'] = pd.to_datetime(train_merged['Date'])
train_merged['Year'] = train_merged['Date'].dt.year
train_merged['Month'] = train_merged['Date'].dt.month

In [9]:
"""
    CompetitionOpenElapsedDays, Promo2ElapsedDays 특징 추출
"""

comp_elapsed_days_lst = []
promo_elapsed_days_lst = []

for k, v in train_merged.iterrows():
    if v.CompetitionOpenSinceYear == 0:  # 경쟁 스토어가 없는 경우 -> 0으로 처리
        comp_elapsed_days = 0
        
    else:  # 경쟁 스토어가 있는 경우 
        # CompetitionOpenElapsedDays 계산: Date - 경쟁 스토어 개업일
        competition_open_date = datetime(  # 경쟁 스토어 개업일
            year=v.CompetitionOpenSinceYear, month=v.CompetitionOpenSinceMonth, day=1)
        comp_elapsed_days = (v.Date - competition_open_date).days \
            if (v.Date - competition_open_date).days > 0 \
            else 0  # Date가 경쟁 스토어 개업일보다 이전일 경우 -> 0으로 처리
    
    comp_elapsed_days_lst.append(comp_elapsed_days)
    
    if v.Promo2SinceYear == 0: # 프로모션하지 않는 스토어 -> 0으로 처리
        promo_elapsed_days = 0
    
    else:
        # Promo2ElapsedDays 계산: Date - 프로모션 시작일
        num_week = int(v.Promo2SinceWeek)  # 프로모션 시작 주차, timedelta 처리를 위해 기본 int형으로 변환
        # 프로모션 시작일
        promo_start_date = datetime(year=v.Promo2SinceYear, month=1, day=1) + relativedelta(weeks=num_week)
        promo_elapsed_days = (v.Date - promo_start_date).days \
            if (v.Date - promo_start_date).days > 0 \
            else 0  # Date가 프로모션 시작일보다 이전일 경우 -> 0으로 처리

    promo_elapsed_days_lst.append(promo_elapsed_days)

train_merged['CompetitionOpenElapsedDays'] = comp_elapsed_days_lst
train_merged['Promo2ElapsedDays'] = promo_elapsed_days_lst

In [10]:
train_merged.iloc[0]

Store                                           1
DayOfWeek                                       5
Date                          2015-07-31 00:00:00
Sales                                        5263
Customers                                     555
Open                                            1
Promo                                           1
StateHoliday                                    0
SchoolHoliday                                   1
StoreType                                       c
Assortment                                      a
CompetitionDistance                          1270
CompetitionOpenSinceMonth                       9
CompetitionOpenSinceYear                     2008
Promo2                                          0
Promo2SinceWeek                                 0
Promo2SinceYear                                 0
PromoInterval                                   0
Year                                         2015
Month                                           7


In [11]:
# 원핫인코딩: DayOfWeek, Year, Month, StoreType, Assortment, PromoInterval
columns = ['DayOfWeek', 'Year', 'Month', 'StoreType', 'Assortment', 'PromoInterval', 'StateHoliday']
train_merged = pd.get_dummies(train_merged, columns=columns)

In [12]:
# 비영업일 데이터 제거
train_merged = train_merged.query('Open != 0')

In [13]:
# 특징 정규화: Promo2ElapsedDays, CompetitionDistance, CompetitionOpenElapsedDays
columns_scaling = ['Promo2ElapsedDays', 'CompetitionDistance', 'CompetitionOpenElapsedDays']

scaler = MinMaxScaler()
train_merged[columns_scaling] = scaler.fit_transform(train_merged[columns_scaling])

### 훈련 데이터셋 준비
+ 대상변수: Sales
+ 특징: StateHoliday, SchoolHoliday, Promo, Promo2ElapsedDays, CompetitionDistance, CompetitionOpenElapsedDays, 나머지 one-hot encoded 특징들

In [16]:
# 특징 순서 재정렬
columns = ['SchoolHoliday', 'Promo', 'Promo2ElapsedDays',
        'CompetitionDistance', 'CompetitionOpenElapsedDays',
        'Year_2013', 'Year_2014', 'Year_2015',
        'Month_1', 'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6',
        'Month_7', 'Month_8', 'Month_9', 'Month_10', 'Month_11', 'Month_12',
        'DayOfWeek_1', 'DayOfWeek_2', 'DayOfWeek_3', 'DayOfWeek_4',
        'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_7', 'StateHoliday_0',
        'StateHoliday_a', 'StateHoliday_b', 'StateHoliday_c',
        'StoreType_a', 'StoreType_b', 'StoreType_c', 'StoreType_d',
        'Assortment_a', 'Assortment_b', 'Assortment_c',
        'PromoInterval_0', 'PromoInterval_Feb,May,Aug,Nov',
        'PromoInterval_Mar,Jun,Sept,Dec','Sales'
       ]

train_merged = train_merged[columns]

In [19]:
# train/valid 데이터셋 준비
train_refined, valid_refined = train_test_split(train_merged, test_size=0.3)

In [22]:
# 훈련 데이터셋 저장
train_refined.to_csv("./data/train_refined.csv", index=False)
valid_refined.to_csv("./data/valid_refined.csv", index=False)